In [4]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

c:\Users\imars\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
label2idx = {'O': 0, 'B-discount': 1, 'B-value': 2, 'I-value': 3}

In [6]:
model_name = 'ai-forever/ruElectra-small'
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels = len(label2idx)).to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\imars\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
import pandas as pd

data = pd.read_csv('data_with_entity.csv')

In [39]:
def tokenize_and_prepare_inputs(texts):
    tokenized_inputs = tokenizer(texts, truncation=True, is_split_into_words=True, return_tensors="pt", padding=True, max_length=512)
    return tokenized_inputs.to('cpu')

# Функция для получения предсказаний
def predict(texts):
    model.eval()
    inputs = tokenize_and_prepare_inputs(texts)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)
    return predictions

In [41]:
tokenized_text = tokenize_and_prepare_inputs(sent)

In [63]:
import time
mean_time = []
for i in range(30):
    v_data = [' '.join(data['sent'].sample().to_list()) for i in range(25)]
    sent = [i.split() for i in v_data]
    start_time = time.time()
    pred = predict(sent)
    end_time = time.time()
    mean_time.append(end_time-start_time)

In [75]:
import numpy as np

print('Среднее время работы: ',np.mean(mean_time), 'среднее квадратичное отклонение: ', np.std(mean_time))

Среднее время работы:  9.194216442108154 среднее квадратичное отклонение:  4.750438888324384


In [1]:
import onnxruntime
from onnxruntime import (
    InferenceSession,
    SessionOptions
)


def create_onnx_session(
        model_path: str,
        provider: str = "CPUExecutionProvider"
) -> InferenceSession:
    """Создание сессии для инференса модели с помощью ONNX Runtime.

    @param model_path: путь к модели в формате ONNX
    @param provider: инференс на ЦП
    @return: ONNX Runtime-сессия
    """    
    options = SessionOptions()
    options.graph_optimization_level = \
        onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
    options.intra_op_num_threads = 1
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    return session

In [2]:
session = create_onnx_session('ruElectra-small-onnx-quantized.onnx')

In [9]:
import numpy as np
from transformers import AutoTokenizer
from onnxruntime import InferenceSession


def onnx_inference(
        text: list,
        session: InferenceSession,
        tokenizer: AutoTokenizer,
        max_length: int
) -> np.ndarray:
    """Инференс модели с помощью ONNX Runtime.

    @param text: входной текст для классификации
    @param session: ONNX Runtime-сессия
    @param tokenizer: токенизатор
    @param max_length: максимальная длина последовательности в токенах
    @return: логиты на выходе из модели
    """
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="np",
    )
    input_feed = {
        "input_ids": inputs["input_ids"].astype(np.int64)
    }
    outputs = session.run(
        output_names=["output"],
        input_feed=input_feed
    )[0]
    return outputs

In [10]:
import time
mean_time_onnx = []
for i in range(30):
    v_data = [' '.join(data['sent'].sample().to_list()) for i in range(25)]
    sent = [i.split() for i in v_data]
    start_time = time.time()
    pred = onnx_inference(v_data, session, tokenizer, 512)
    end_time = time.time()
    mean_time_onnx.append(end_time-start_time)

In [13]:
torch.argmax(torch.Tensor(pred), dim=2)

tensor([[3, 1, 1,  ..., 1, 1, 1],
        [3, 1, 1,  ..., 1, 1, 1],
        [3, 1, 1,  ..., 1, 1, 1],
        ...,
        [3, 1, 1,  ..., 1, 1, 1],
        [3, 1, 1,  ..., 1, 1, 1],
        [3, 1, 1,  ..., 1, 1, 1]])

In [11]:
import numpy as np

print('Среднее время работы: ',np.mean(mean_time_onnx), 'среднее квадратичное отклонение: ', np.std(mean_time_onnx))

Среднее время работы:  3.8348785161972048 среднее квадратичное отклонение:  0.722076557095312
